In [ ]:
#1

In [1]:
import os
import shutil
import pyinputplus as pyip
import sys
import re

In [2]:
def is_path_valid(path):
    try:
        os.path.abspath(path)
    except Exception:
        'Invalid path, try again'

In [3]:
def copy_extension():
    extension = input('Enter an extension (ex: \'txt\'):\n')
    
    search_folder = pyip.inputCustom(is_path_valid, '\nEnter the path of the directory to search: ')
    search_folder = os.path.abspath(search_folder)
    
    if not os.path.exists(search_folder):
        print('Path not found')
        return
    
    yes_no = pyip.inputYesNo('\nCopying from: ' + search_folder + '\nWould you like to proceed? Enter (y)es or (n)o: ')
    if yes_no == 'no':
        return
    print()
    
    files_to_copy = []
    for foldername, subfolders, filenames in os.walk(search_folder):
        for a_file in filenames:
            if a_file.endswith(extension):
                file_to_copy = os.path.relpath(os.path.join(foldername, a_file), search_folder)
                print('File found: ' + file_to_copy)
                files_to_copy.append(file_to_copy)
    yes_no = pyip.inputYesNo('\nWould you like to proceed? Enter (y)es or (n)o: ')
    
    new_folder = pyip.inputCustom(is_path_valid, '\nEnter the path of the directory to be copied to: ')
    new_folder = os.path.abspath(new_folder)
    
    yes_no = pyip.inputYesNo('\nCopying to: ' + new_folder + '\nWould you like to proceed? Enter (y)es or (n)o: ')
    if yes_no == 'no':
        return
    
    os.makedirs(new_folder)
    for a_file in files_to_copy:
        shutil.copy(a_file, new_folder)


In [ ]:
copy_extension()

In [ ]:
#2

In [4]:
def get_subfolder_bytes(path):
    total_size = 0
    for foldername, subfolders, filenames in os.walk(path):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            if not os.path.islink(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

In [ ]:
target_size = pyip.inputNum('\nEnter the minimum number of MB to flag: ', min=0)

while True:
    search_path = pyip.inputCustom(is_path_valid, 'Enter the path of the directory to be searched: ')
    if not os.path.exists(search_path):
        print('Path not found\n')
        continue
    search_path = os.path.abspath(search_path)

    yes_no = pyip.inputYesNo('\n' + search_path + '\nIs the search path correct? Enter (y)es or (n)o: ')
    if yes_no == 'yes':
        break
    print()

print()
for foldername, subfolders, filenames in os.walk(search_path):
    print_folder_name = False
    for subfolder in subfolders:
        folder_size = get_subfolder_bytes(os.path.join(foldername, subfolder)) * 9.537e-7
        if folder_size >= target_size:
            if not print_folder_name:
                print_folder_name = True
                print(f'{foldername}...')
            print('\tFolder flagged: {:} is {:.2f} MB'.format(os.path.relpath(os.path.join(foldername, subfolder), foldername), folder_size))
    for filename in filenames:
        file_size = os.path.getsize(os.path.join(foldername, filename)) * 9.537e-7
        if file_size >= target_size:
            if not print_folder_name:
                print_folder_name = True
                print(f'{foldername}...')
            print('\tFile flagged: {:} is {:.2f} MB'.format(os.path.relpath(os.path.join(foldername, filename), foldername), file_size))
    if print_folder_name:
        print()

In [ ]:
#3

In [2]:
def is_dir_valid(path):
    if not os.path.isdir(path):
        raise Exception('Directory not found')

In [3]:
def left_pad_integer(num, length, char='0'):
    num = str(num)
    if len(num) > length:
        raise Exception('The new file number would exceed the specified length (ex: spam100.txt when you specified the number should be length 2)')
    while len(num) < length:
        num = char + num
    return num

In [4]:
def get_number_from_regex_match(match):
    return int(match[1])

In [5]:
def get_filename_regex(prefix, extension):
    return re.compile('^' + prefix + r'(\d+)\.' + extension + '$')

In [6]:
def close_file_number_gap(search_path, prefix, extension, padded_file_number_length):    
    filename_regex = get_filename_regex(prefix, extension)
    
    files_to_rename = []
    for filename in os.listdir(search_path):
        if os.path.isfile(os.path.join(search_path, filename)):
            match = filename_regex.search(filename)
            if match is not None:
                files_to_rename.append(match)
    if len(files_to_rename) == 0:
        print('\nNo matches found given specified prefix and extension.')
        return
        
    files_to_rename.sort(key=get_number_from_regex_match)
    
    print('\nThe following renamings will occur:')
    rename_dict = {}
    for i, match in enumerate(files_to_rename):
        rename_dict[match[0]] = prefix + left_pad_integer(i + 1, padded_file_number_length) + '.' + extension
        print('    ' + match[0] + '    ->    ' + rename_dict[match[0]])
        

    yes_no = pyip.inputYesNo('\nWould you like to proceed? Enter (y)es or (n)o: ')
    if yes_no == 'no':
        return
    
    for key in rename_dict.keys():
        shutil.move(os.path.join(search_path, key), os.path.join(search_path, rename_dict[key]))
        
    print('\nDone.')
    return rename_dict.values()

In [7]:
def insert_file_number_gap(search_path, prefix, extension, padded_file_number_length, current_files):
    filename_regex = get_filename_regex(prefix, extension)
    current_matches = []
    for filename in current_files:
        match = filename_regex.search(filename)
        if match is not None:
            current_matches.append(match)
    
    current_matches.sort(key=get_number_from_regex_match)
    print('    -> 0.')
    for i, match in enumerate(current_matches):
        print('    -> ' + str(i + 1) + '. ' + match[0])
        
    gap_index = pyip.inputInt('\nEnter the number corresponding to the file you would like to insert the gap AFTER: ', min=0, max=len(current_files))
    gap_amount = pyip.inputInt('\nEnter the number of blanks you would like to insert (ex: \'2\' to skip from spam03 to spam06)', min=0)
    
    print('\nThe following renamings will occur:')
    rename_dict = {}
    for i, match in enumerate(current_matches):
        if i < gap_index:
            rename_dict[match[0]] = match[0]
        else:
            rename_dict[match[0]] = prefix + left_pad_integer(int(match[1]) + gap_amount, padded_file_number_length) + '.' + extension
        print('    ' + match[0] + '    ->    ' + rename_dict[match[0]])
    
    yes_no = pyip.inputYesNo('\nWould you like to proceed? Enter (y)es or (n)o: ')
    if yes_no == 'no':
        return
    
    for key in rename_dict.keys():
        shutil.move(os.path.join(search_path, key), os.path.join(search_path, rename_dict[key]))
        
    print('\nDone.')
    return rename_dict.values()

In [8]:
def main_3():
    search_path = pyip.inputCustom(is_dir_valid, '\nEnter the path of the directory to be searched: ')
    search_path = os.path.abspath(search_path)
    
    yes_no = pyip.inputYesNo('\n' + search_path + '\nIs the search path correct? Enter (y)es or (n)o: ')
    if yes_no == 'no':
        return
    
    print('\nFiles found:')
    file_count = 0
    for filename in os.listdir(search_path):
        if os.path.isfile(os.path.join(search_path, filename)):
            print('    -> ' + filename)
            file_count += 1
    if file_count == 0:
        print('    No files found.')
        return
    
    prefix = input('\nEnter the prefix of the files to be renamed: ')
    extension = input('\nEnter the extension of the files to be renamed (ex: \'txt\'): ')
    padded_file_number_length = pyip.inputInt('\nEnter the length the file number should be after left padding with zeroes (ex: for 012 enter \'3\'): ', min=1)
    
    new_names = close_file_number_gap(search_path, prefix, extension, padded_file_number_length)
    
    while True:
        if new_names == None:
            return
        yes_no = pyip.inputYesNo('\nWould you like to insert a gap in the file numbers? Enter (y)es or (n)o: ')
        if yes_no == 'no':
            return
        new_names = insert_file_number_gap(search_path, prefix, extension, padded_file_number_length, new_names)

In [9]:
main_3()


Enter the path of the directory to be searched: try e

C:\Users\jacob\Documents\CompSci\PythonProjects\Automate the Boring Stuff\try e
Is the search path correct? Enter (y)es or (n)o: y

Files found:
    -> hat1.txt
    -> hat2.txt
    -> hat45.txt

Enter the prefix of the files to be renamed: hat

Enter the extension of the files to be renamed (ex: 'txt'): txt

Enter the length the file number should be after left padding with zeroes (ex: for 012 enter '3'): 3

The following renamings will occur:
    hat1.txt    ->    hat001.txt
    hat2.txt    ->    hat002.txt
    hat45.txt    ->    hat003.txt

Would you like to proceed? Enter (y)es or (n)o: y

Done.

Would you like to insert a gap in the file numbers? Enter (y)es or (n)o: y
    -> 0.
    -> 1. hat001.txt
    -> 2. hat002.txt
    -> 3. hat003.txt

Enter the number corresponding to the file you would like to insert the gap AFTER: 1

Enter the number of blanks you would like to insert (ex: '2' to skip from spam03 to spam06)11

The fol